In [36]:
import time
import numpy as np
import pandas as pd
import time
import pickle
import sys
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [37]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [38]:
## Windows

# driver = webdriver.Chrome( )
# driver.get('https:/1xbet.whoscored.com/')
# driver.implicitly_wait(3)
# driver.close()

In [39]:
## MAC
#driver = webdriver.Chrome()
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get('https:/1xbet.whoscored.com/')
driver.implicitly_wait(3)
driver.close()

In [ ]:
def crawling_league_teams(region, tournaments, api_delay_term=5):

    """
    cawling league team_id and team name datas
    
    Arg :
        region : region id (from whoscored) for certain league ex) england --- 252, spain --- 206
        tournaments : tournament(or league) id ex) premier league --- 2,
    
    return :
        crawling league team_id, team_name datas belong team_id parameter
        return pandas dataframe columns=team_id, team_name
    

    """
    
    # connect webdriver
    url = "https://1xbet.whoscored.com/Regions/" + str(region) + "/Tournaments/" + str(tournaments)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)

    # wait get league team datas
    time.sleep(api_delay_term) 
    
    # make pandas dataframe
    team_df = pd.DataFrame(columns=["team_id","team_name"])

    # get team data
    teams = driver.find_element(By.XPATH, "//*[@id='standings-20934-content']")
    teams = teams.find_elements(By.CSS_SELECTOR, "a.team-link")
    
    for team in teams:
        team_name = team.get_attribute("innerHTML")
        team_id = team.get_attribute("href").split("/")[4]
        team_df.loc[len(team_df)] = {"team_id":team_id, "team_name":team_name }
        
    # close webdriver
    driver.close()
    
    return team_df

- Premier League teams and id.

In [156]:
PL_df = crawling_league_teams(252, 2, api_delay_term=3)
PL_df

,team_id,team_name
0,13,Arsenal
1,167,Manchester City
2,23,Newcastle
3,32,Manchester United
4,30,Tottenham
5,26,Liverpool
6,170,Fulham
7,211,Brighton
8,189,Brentford
9,15,Chelsea


In [16]:
def crawling_player_summary(team_id, api_delay_term=5):
    """
    crawling player statistics of certain team
    
    Args :
        team_id : team number 
        
    return :
        player statistics (dataframe)
    
    """

    # connect webdriver
    url = "https://www.whoscored.com/Teams/" + str(team_id)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)

    # wait for getting data
    time.sleep(api_delay_term)
    
    # make pandas dataframe
    player_summary_df = pd.DataFrame(columns=[
        "player_number", "name","age","position","tall","weight","games",
        "mins", "goals", "asists", "yel", "red", "spg", "ps","aw","motm", "rating"
        ])
    
    # get player summay datas
    elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")
    for element in elements:
        
        player_dict = {
            "player_number": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, ("a"))[0].get_attribute("href").split("/")[4], 
            "name": element.find_element(By.CSS_SELECTOR, "td > a > span").get_attribute("textContent").split("\t")[0],     
            "age": element.find_element(By.CSS_SELECTOR, "td > span > span:nth-child(1)").get_attribute("textContent").split("\t")[0],
            "position": element.find_element(By.CSS_SELECTOR, "td > span > span:nth-child(2)").get_attribute("textContent").split("\t")[0][2:], 
            "tall": element.find_elements(By.CSS_SELECTOR, ("td"))[2].get_attribute("textContent").split("\t")[0],
            "weight": element.find_elements(By.CSS_SELECTOR, ("td"))[3].get_attribute("textContent").split("\t")[0], 
            "games": element.find_elements(By.CSS_SELECTOR, ("td"))[4].get_attribute("textContent").split("\t")[0], 
            "mins": element.find_elements(By.CSS_SELECTOR, ("td"))[5].get_attribute("textContent").split("\t")[0],
            "goals": element.find_elements(By.CSS_SELECTOR, ("td"))[6].get_attribute("textContent").split("\t")[0],
            "asists": element.find_elements(By.CSS_SELECTOR, ("td"))[7].get_attribute("textContent").split("\t")[0],
            "yel": element.find_elements(By.CSS_SELECTOR, ("td"))[8].get_attribute("textContent").split("\t")[0],
            "red": element.find_elements(By.CSS_SELECTOR, ("td"))[9].get_attribute("textContent").split("\t")[0],
            "spg": element.find_elements(By.CSS_SELECTOR, ("td"))[10].get_attribute("textContent").split("\t")[0],
            "ps": element.find_elements(By.CSS_SELECTOR, ("td"))[11].get_attribute("textContent").split("\t")[0],
            "aw": element.find_elements(By.CSS_SELECTOR, ("td"))[12].get_attribute("textContent").split("\t")[0],
            "motm": element.find_elements(By.CSS_SELECTOR, ("td"))[13].get_attribute("textContent").split("\t")[0],
            "rating": element.find_elements(By.CSS_SELECTOR, ("td"))[14].get_attribute("textContent").split("\t")[0],
        }
        
        player_summary_df.loc[len(player_summary_df)] = player_dict
        
    # close webdriver
    driver.close()
    player_summary_df.replace("-", "0", inplace=True)
    return player_summary_df

- Player stat table (Liverpool) 

In [158]:
player_liverpool = crawling_player_summary(26, api_delay_term=5)

In [159]:
player_liverpool

,player_number,name,age,position,tall,weight,games,mins,goals,asists,yel,red,spg,ps,aw,motm,rating
0,377168,Luis Díaz,25,AM(L),178,65,7(1),615,3,2,2,0,2,81,0.8,1,7.28
1,96182,Roberto Firmino,31,"M(CLR),FW",181,76,11(2),937,7,3,0,0,2.5,79.6,1,2,7.20
2,108226,Mohamed Salah,30,"AM(CLR),FW",175,71,17,1509,7,4,0,0,3.4,76.5,0.1,1,7.01
3,114147,Alisson,30,GK,193,91,17,1530,0,1,0,0,0,81.7,0.2,2,6.99
4,400828,Darwin Núñez,23,"AM(CL),FW",187,81,10(3),874,5,2,1,1,4.2,69.5,1.5,2,6.91
5,74939,Thiago,31,M(C),174,70,11,893,0,0,2,0,0.7,86.4,1,1,6.91
6,318871,Trent Alexander-Arnold,24,"D(R),M(R)",175,72,15(2),1249,1,1,2,0,1,77.2,0.6,0,6.90
7,115726,Andrew Robertson,28,"D(L),M(L)",178,64,12(3),1061,0,5,0,0,0.3,83.9,0.6,0,6.80
8,95408,Virgil van Dijk,31,D(C),195,92,17,1486,2,0,1,0,1.1,89.1,2.6,0,6.79
9,115916,Fabinho,29,"D(CR),DMC",188,78,13(3),1178,0,0,2,0,0.4,88.5,0.9,0,6.69


In [61]:
# Premier league 1920 plyaer stat table(URL)
URL = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/PlayerStatistics/England-Premier-League-2022-2023'

In [67]:
from selenium.webdriver.common.action_chains import ActionChains

url = str(URL)
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(url)

time.sleep(3)

ac = ActionChains(driver)
ac.move_by_offset(0,0)
ac.click()
ac.perform()

In [59]:
def crawling_player_statistics(URL):
    """
    crawling player statistics of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)
    tabs = driver.window_handles

    time.sleep(3)

    while len(tabs) != 1:
        driver.switch_to_window(tabs[1])
        driver.close()

    driver.switch_to_window(tabs[0])
    
    time.sleep(3)
    
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-summary > div > dl.listbox.right > dt > b')[0].get_attribute("textContent").split("\\")[0]
    page = int(page.split('/')[1].split(' ')[0])
    player_statistics_df = pd.DataFrame(columns=[
        "player_name", "team_number", "team_name", "Apps", "Mins", "Goals",
        "Assists", "Yel", "Red", "SpG","PS%", "AerialsWon","MoM","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.ID, 'next').click()
                time.sleep(4)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "Apps", "Mins", "Goals",
                "Assists", "Yel", "Red", "SpG","PS%", "AerialsWon","MoM","Rating"])
            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, ("a"))[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Goals": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Assists": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "Yel": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "Red": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "SpG": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "PS%": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "AerialsWon": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "MoM": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict

            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
            
    driver.close()
    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)
    

In [60]:
PL_player_statistics_2223 = crawling_player_statistics(URL)

AttributeError: 'WebDriver' object has no attribute 'switch_to_window'

In [163]:
PL_player_statistics_2223.head()

,player_name,team_number,team_name,Apps,Mins,Goals,Assists,Yel,Red,SpG,PS%,AerialsWon,MoM,Rating
0,Erling Haaland,167,Man City,15(1),1311,21,3,3,0,4.1,75.7,1,4,7.79
1,Kieran Trippier,23,Newcastle,18,1597,1,4,4,0,0.4,75.9,1.3,7,7.70
2,Kevin De Bruyne,167,Man City,16(1),1405,3,9,1,0,2.6,79.9,0.4,5,7.68
3,Gabriel Jesus,13,Arsenal,14,1226,5,5,4,0,3.6,81.9,1.7,2,7.59
4,Harry Kane,30,Tottenham,18,1608,15,1,4,0,3.6,71.7,1.7,3,7.55


In [164]:
#PL_player_statistics_2223.to_csv('PL_player_summary_2223.csv')

In [49]:
def crawling_player_statistics_defensive(URL):
    """
    crawling player statistics(defensive) of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)
    
    time.sleep(3)

    driver.find_element(By.ID, 'next').click()

    time.sleep(3)

    driver.find_element(By.ID, 'first').click()
    
    time.sleep(3)
    
    driver.find_element(By.LINK_TEXT, "Defensive").click()
    
    time.sleep(3)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-defensive > div > dl.listbox.right > dt > b')[0].get_attribute("textContent")
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Tackles",
            "Inter", "Fouls", "Offsides", "Clear","Drb", "Blocks","OwnG","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(4)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Tackles",
                "Inter", "Fouls", "Offsides", "Clear","Drb", "Blocks","OwnG","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[4].get_attribute("textContent").split("\t")[0],
                    "position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Tackles": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Inter": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "Fouls": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "Offsides": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Clear": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "Drb": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "Blocks": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "OwnG": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()
    
    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)
    

In [166]:
PL_player_defensive_2223 = crawling_player_statistics_defensive(URL)

page 30: 100%|██████████| 30/30 [03:53<00:00,  7.78s/it]


In [167]:
PL_player_defensive_2223.head()

,player_name,team_number,team_name,age,position,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating
0,Erling Haaland,167,Man City,22,FW,15(1),1311,0.1,0.1,0.8,0,0.4,0.1,0.1,0,7.79
1,Kieran Trippier,23,Newcastle,32,"D(LR),M(R)",18,1597,2.2,1.4,1,0.3,1.7,1,0.6,0,7.70
2,Kevin De Bruyne,167,Man City,31,"M(CLR),FW",16(1),1405,1.1,0.2,0.9,0,0.4,0.8,0.1,0,7.68
3,Gabriel Jesus,13,Arsenal,25,"AM(LR),FW",14,1226,1.4,0.6,1.7,0,0.6,0.6,0.1,0,7.59
4,Harry Kane,30,Tottenham,29,"AM(C),FW",18,1608,0.3,0.1,0.7,0.1,0.5,0.2,0.2,0,7.55


In [168]:
#PL_player_defensive_2223.to_csv('PL_player_defensive_2223.csv')

In [50]:
def crawling_player_statistics_offensive(URL):
    """
    crawling player statistics(offensive) of certain league and season.
    
    Args :
        URL : player statistics webpage URL
        
    return :
        player statistics (dataframe)
    
    """
    url = str(URL)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)
    
    time.sleep(3)

    driver.find_element(By.ID, 'next').click()

    time.sleep(3)

    driver.find_element(By.ID, 'first').click()
    
    time.sleep(3)
    
    driver.find_element(By.LINK_TEXT, "Offensive").click()
    
    time.sleep(3)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-offensive > div > dl.listbox.right > dt > b')[0].get_attribute("textContent").split("\t")[0]
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Goals",
            "Assists", "SpG", "KeyP", "Dribble","Fouled", "Off","Disp","UnsTch","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(4)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Goals",
                "Assists", "SpG", "KeyP", "Dribble","Fouled", "Off","Disp","UnsTch","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR,"a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR,"span")[4].get_attribute("textContent").split("\t")[0],
                    "position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Goals": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "Assists": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "SpG": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "KeyP": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Dribble": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "Fouled": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "Off": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "Disp": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                    "UnsTch": element.find_elements(By.CSS_SELECTOR, "td")[12].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[13].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()

    player_statistics_df.replace("-", "0", inplace=True)
    return(player_statistics_df)

In [51]:
#PL_player_offensive_2223 = crawling_player_statistics_offensive(pl_url)

In [171]:
PL_player_offensive_2223.head()

,player_name,team_number,team_name,age,position,Apps,Mins,Goals,Assists,SpG,KeyP,Dribble,Fouled,Off,Disp,UnsTch,Rating
0,Erling Haaland,167,Man City,22,FW,15(1),1311,21,3,4.1,1,0.3,0.7,0.3,0.9,1.5,7.79
1,Kieran Trippier,23,Newcastle,32,"D(LR),M(R)",18,1597,1,4,0.4,2.6,0.3,0.7,0.2,0.1,0.6,7.70
2,Kevin De Bruyne,167,Man City,31,"M(CLR),FW",16(1),1405,3,9,2.6,3.5,0.8,0.8,0.2,1.2,1.5,7.68
3,Gabriel Jesus,13,Arsenal,25,"AM(LR),FW",14,1226,5,5,3.6,1.6,1.9,2.7,0.6,2.9,3.9,7.59
4,Harry Kane,30,Tottenham,29,"AM(C),FW",18,1608,15,1,3.6,1.7,0.8,1.4,0.3,2,1.7,7.55


In [179]:
#PL_player_offensive_2223.to_csv('PL_player_offensive_2223.csv')

In [52]:
def crawling_player_statistics_passing(URL):
    url = str(URL)
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)
    
    time.sleep(3)

    driver.find_element(By.ID, 'next').click()

    time.sleep(3)

    driver.find_element(By.ID, 'first').click()
    
    time.sleep(3)
    
    driver.find_element(By.LINK_TEXT, "Passing").click()
    
    time.sleep(3)
    page = driver.find_elements(By.CSS_SELECTOR, '#statistics-paging-passing > div > dl.listbox.right > dt > b')[0].get_attribute("textContent")
    page = int(page.split('/')[1].split(' ')[0])
    
    player_statistics_df = pd.DataFrame(columns=[
            "player_name", "team_number", "team_name", "age","position", "Apps", "Mins", "Assists",
            "KeyP", "AvgP","PS%", "Crosses","LongB","ThrB","Rating"])
    
    with tqdm(total=page, file=sys.stdout) as pbar :
        for i in range(page):
            if i != 0:
                driver.find_element(By.LINK_TEXT, 'next').click()
                time.sleep(4)

            temp_df = pd.DataFrame(columns=[
                "player_name", "team_number", "team_name", "age","position", "Apps", "Mins","Assists",
                "KeyP", "AvgP","PS%", "Crosses","LongB","ThrB","Rating"])

            elements = driver.find_elements(By.CSS_SELECTOR, "#player-table-statistics-body tr")[10:]

            for element in elements:
                player_statistics_dict = { 
                    "player_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[0].
                    find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0],
                    "team_number": element.find_elements(By.CSS_SELECTOR, "td")[0].
                    find_elements(By.CSS_SELECTOR, "a")[1].get_attribute("href").split("/")[4],
                    "team_name": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "a")[1].find_elements(By.CSS_SELECTOR, "span")[0].get_attribute("textContent").split("\t")[0].split(",")[0],
                    "age": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[4].get_attribute("textContent").split("\t")[0],
                    "position": element.find_elements(By.CSS_SELECTOR, "td")[0].find_elements(By.CSS_SELECTOR, "span")[5].get_attribute("textContent").split("\t")[0][2:],
                    "Apps": element.find_elements(By.CSS_SELECTOR, "td")[2].get_attribute("textContent").split("\t")[0],
                    "Mins": element.find_elements(By.CSS_SELECTOR, "td")[3].get_attribute("textContent").split("\t")[0], 
                    "Assists": element.find_elements(By.CSS_SELECTOR, "td")[4].get_attribute("textContent").split("\t")[0],
                    "KeyP": element.find_elements(By.CSS_SELECTOR, "td")[5].get_attribute("textContent").split("\t")[0], 
                    "AvgP": element.find_elements(By.CSS_SELECTOR, "td")[6].get_attribute("textContent").split("\t")[0], 
                    "PS%": element.find_elements(By.CSS_SELECTOR, "td")[7].get_attribute("textContent").split("\t")[0],
                    "Crosses": element.find_elements(By.CSS_SELECTOR, "td")[8].get_attribute("textContent").split("\t")[0],
                    "LongB": element.find_elements(By.CSS_SELECTOR, "td")[9].get_attribute("textContent").split("\t")[0],
                    "ThrB": element.find_elements(By.CSS_SELECTOR, "td")[10].get_attribute("textContent").split("\t")[0],
                    "Rating": element.find_elements(By.CSS_SELECTOR, "td")[11].get_attribute("textContent").split("\t")[0],
                }
                temp_df.loc[len(temp_df)] = player_statistics_dict


            player_statistics_df = pd.concat([player_statistics_df,temp_df])
            pbar.update(1)
            pbar.set_description('page {}'.format(i+1))
        
    driver.close()
    player_statistics_df.replace("-", "0", inplace=True)

    return(player_statistics_df)

In [174]:
PL_player_passing_2223 = crawling_player_statistics_passing(URL)

page 30: 100%|██████████| 30/30 [03:19<00:00,  6.64s/it]


In [175]:
PL_player_passing_2223.head()

,player_name,team_number,team_name,age,position,Apps,Mins,Assists,KeyP,AvgP,PS%,Crosses,LongB,ThrB,Rating
0,Erling Haaland,167,Man City,22,FW,15(1),1311,3,1,14.1,75.7,0,0.1,0.1,7.79
1,Kieran Trippier,23,Newcastle,32,"D(LR),M(R)",18,1597,4,2.6,50.2,75.9,3.7,3.5,0.3,7.70
2,Kevin De Bruyne,167,Man City,31,"M(CLR),FW",16(1),1405,9,3.5,44.9,79.9,2.4,2.9,0.5,7.68
3,Gabriel Jesus,13,Arsenal,25,"AM(LR),FW",14,1226,5,1.6,27.6,81.9,0.1,0.5,0.1,7.59
4,Harry Kane,30,Tottenham,29,"AM(C),FW",18,1608,1,1.7,22.4,71.7,0.2,1.7,0.1,7.55


In [178]:
PL_player_passing_2223.to_csv('PL_player_passing_2223.csv')

In [180]:
passing=pd.read_csv('PL_player_passing_2223.csv')
summary=pd.read_csv('PL_player_summary_2223.csv')
offensive=pd.read_csv('PL_player_offensive_2223.csv')
defensive=pd.read_csv('PL_player_defensive_2223.csv')

In [181]:
PL_player_2223 = pd.merge(pd.merge(passing,summary), pd.merge(offensive,defensive))

In [182]:
PL_player_2223.head()

,Unnamed: 0,player_name,team_number,team_name,age,position,Apps,Mins,Assists,KeyP,...,Disp,UnsTch,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG
0,0,Erling Haaland,167,Man City,22,FW,15(1),1311,3,1.0,...,0.9,1.5,0.1,0.1,0.8,0.0,0.4,0.1,0.1,0
1,1,Kieran Trippier,23,Newcastle,32,"D(LR),M(R)",18,1597,4,2.6,...,0.1,0.6,2.2,1.4,1.0,0.3,1.7,1.0,0.6,0
2,2,Kevin De Bruyne,167,Man City,31,"M(CLR),FW",16(1),1405,9,3.5,...,1.2,1.5,1.1,0.2,0.9,0.0,0.4,0.8,0.1,0
3,3,Gabriel Jesus,13,Arsenal,25,"AM(LR),FW",14,1226,5,1.6,...,2.9,3.9,1.4,0.6,1.7,0.0,0.6,0.6,0.1,0
4,4,Harry Kane,30,Tottenham,29,"AM(C),FW",18,1608,1,1.7,...,2.0,1.7,0.3,0.1,0.7,0.1,0.5,0.2,0.2,0


# crwaling player stats at once

In [53]:
def crwaling_player_stats_at_once(URL):
    stats_summary = crawling_player_statistics(URL)
    print('player stats summary : done')
    stats_defensive = crawling_player_statistics_defensive(URL)
    print('player stats defensive : done')
    stats_offensive = crawling_player_statistics_offensive(URL)
    print('player stats offensive : done')
    stats_passing = crawling_player_statistics_passing(URL)
    print('player stats pasing : done')
    player_stats = pd.merge(pd.merge(stats_passing, stats_summary), 
                            pd.merge(stats_offensive, stats_defensive))
    return(player_stats)

In [29]:
# PL2022-23
URL = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/PlayerStatistics/England-Premier-League-2022-2023'

In [151]:
PL_2223 = crwaling_player_stats_at_once(URL)

page 30: 100%|██████████| 30/30 [03:08<00:00,  6.28s/it]
player stats summary : done
page 30: 100%|██████████| 30/30 [03:32<00:00,  7.08s/it]
player stats defensive : done
page 30: 100%|██████████| 30/30 [04:48<00:00,  9.60s/it]
player stats offensive : done
page 30: 100%|██████████| 30/30 [04:06<00:00,  8.23s/it]
player stats pasing : done


In [152]:
PL_2223.head()

,AvgP,PS%,Crosses,LongB,ThrB,Yel,Red,AerialsWon,MoM,player_name,...,UnsTch,Rating,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG


In [36]:
PL_2223.to_csv('PL_player_2223.csv')

In [40]:
#URL1516 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/5826/Stages/12496/PlayerStatistics/England-Premier-League-2015-2016'
#URL1617 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/6335/Stages/13796/PlayerStatistics/England-Premier-League-2016-2017'
#URL1718 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/6829/Stages/15151/PlayerStatistics/England-Premier-League-2017-2018'
#URL1819 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/7361/Stages/16368/PlayerStatistics/England-Premier-League-2018-2019'
#URL1920 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/7811/Stages/17590/PlayerStatistics/England-Premier-League-2019-2020'
URL2021 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8228/Stages/18685/PlayerStatistics/England-Premier-League-2020-2021'
URL2122 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/8618/Stages/19793/PlayerStatistics/England-Premier-League-2021-2022'
URL2223 = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/PlayerStatistics/England-Premier-League-2022-2023'

In [56]:
#PL_1516 = crwaling_player_stats_at_once(URL1516)
#PL_1617 = crwaling_player_stats_at_once(URL1617)
#PL_1718 = crwaling_player_stats_at_once(URL1718)
#PL_1819 = crwaling_player_stats_at_once(URL1819)
#PL_2021 = crwaling_player_stats_at_once(URL2021)
PL_2122 = crwaling_player_stats_at_once(URL2122)
PL_2223 = crwaling_player_stats_at_once(URL2223)

page 1:   3%|▎         | 1/30 [00:01<00:30,  1.04s/it]


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=119.0.6045.199)
Stacktrace:
0   chromedriver                        0x0000000100cf6004 chromedriver + 4169732
1   chromedriver                        0x0000000100cedff8 chromedriver + 4136952
2   chromedriver                        0x0000000100943348 chromedriver + 291656
3   chromedriver                        0x0000000100989ef0 chromedriver + 581360
4   chromedriver                        0x000000010097e4f8 chromedriver + 533752
5   chromedriver                        0x000000010097ddb4 chromedriver + 531892
6   chromedriver                        0x00000001009c3818 chromedriver + 817176
7   chromedriver                        0x000000010097c5e8 chromedriver + 525800
8   chromedriver                        0x000000010097d4b8 chromedriver + 529592
9   chromedriver                        0x0000000100cbc334 chromedriver + 3932980
10  chromedriver                        0x0000000100cc0970 chromedri

In [13]:
PL_1718 = crwaling_player_stats_at_once(URL1718)

page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [02:29<00:00,  5.17s/it]
player stats summary : done
page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [02:36<00:00,  5.40s/it]
player stats defensive : done
page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [02:38<00:00,  5.48s/it]
player stats offensive : done
page 29: 100%|█████████████████████████████████████████████████████████████████████████| 29/29 [02:28<00:00,  5.11s/it]
player stats pasing : done


In [55]:
PL_2021.to_csv('PL_player_2021.csv')

In [44]:
PL_1516.to_csv('PL_player_1516.csv')
PL_1617.to_csv('PL_player_1617.csv')
PL_1718.to_csv('PL_player_1718.csv')
PL_1819.to_csv('PL_player_1819.csv')

In [41]:
## URLs
# PL
pl_url = 'https://1xbet.whoscored.com/Regions/252/Tournaments/2/Seasons/9075/Stages/20934/PlayerStatistics/England-Premier-League-2022-2023'
# BUNDES
budnes_url = 'https://1xbet.whoscored.com/Regions/81/Tournaments/3/Seasons/9120/Stages/21026/PlayerStatistics/Germany-Bundesliga-2022-2023'
# LIGA
liga_url = 'https://1xbet.whoscored.com/Regions/206/Tournaments/4/Seasons/9149/Stages/21073/PlayerStatistics/Spain-LaLiga-2022-2023'
# SERIE
serie_url = 'https://1xbet.whoscored.com/Regions/108/Tournaments/5/Seasons/9159/Stages/21087/PlayerStatistics/Italy-Serie-A-2022-2023'
# ligue1
ligue1_url = 'https://1xbet.whoscored.com/Regions/74/Tournaments/22/Seasons/9129/Stages/21037/PlayerStatistics/France-Ligue-1-2022-2023'

## 

In [45]:
PL_2223 = crwaling_player_stats_at_once(pl_url)

page 31: 100%|██████████| 31/31 [02:03<00:00,  3.98s/it]
player stats summary : done
page 31: 100%|██████████| 31/31 [01:53<00:00,  3.68s/it]
player stats defensive : done
page 31: 100%|██████████| 31/31 [01:56<00:00,  3.77s/it]
player stats offensive : done
page 31: 100%|██████████| 31/31 [01:52<00:00,  3.62s/it]
player stats pasing : done


In [47]:
PL_2223.to_csv('./2223_player_statistics/PL_player_2223.csv')

In [48]:
BUNDES_2223 = crwaling_player_stats_at_once(budnes_url)

page 29: 100%|██████████| 29/29 [01:55<00:00,  3.99s/it]
player stats summary : done
page 29: 100%|██████████| 29/29 [01:45<00:00,  3.65s/it]
player stats defensive : done
page 29: 100%|██████████| 29/29 [01:47<00:00,  3.72s/it]
player stats offensive : done
page 29: 100%|██████████| 29/29 [01:44<00:00,  3.61s/it]
player stats pasing : done


In [51]:
LIGA_2223 = crwaling_player_stats_at_once(liga_url)

page 35: 100%|██████████| 35/35 [02:20<00:00,  4.02s/it]
player stats summary : done
page 35: 100%|██████████| 35/35 [02:08<00:00,  3.66s/it]
player stats defensive : done
page 35: 100%|██████████| 35/35 [02:10<00:00,  3.72s/it]
player stats offensive : done
page 35: 100%|██████████| 35/35 [02:05<00:00,  3.59s/it]
player stats pasing : done


In [54]:
SERIE_2223 = crwaling_player_stats_at_once(serie_url)

page 33: 100%|██████████| 33/33 [02:11<00:00,  3.99s/it]
player stats summary : done
page 33: 100%|██████████| 33/33 [02:00<00:00,  3.66s/it]
player stats defensive : done
page 33: 100%|██████████| 33/33 [02:03<00:00,  3.73s/it]
player stats offensive : done
page 33: 100%|██████████| 33/33 [01:58<00:00,  3.60s/it]
player stats pasing : done


In [56]:
LIGUE1_2223 = crwaling_player_stats_at_once(ligue1_url)

page 32: 100%|██████████| 32/32 [02:06<00:00,  3.96s/it]
player stats summary : done
page 32: 100%|██████████| 32/32 [01:55<00:00,  3.62s/it]
player stats defensive : done
page 32: 100%|██████████| 32/32 [01:58<00:00,  3.71s/it]
player stats offensive : done
page 32: 100%|██████████| 32/32 [01:55<00:00,  3.60s/it]
player stats pasing : done


In [50]:
BUNDES_2223.to_csv('./2223_player_statistics/Bundesliga_player_2223.csv')

In [53]:
LIGA_2223.to_csv('./2223_player_statistics/Laliga_player_2223.csv')

In [55]:
SERIE_2223.to_csv('./2223_player_statistics/Serie_player_2223.csv')

In [59]:
LIGUE1_2223.to_csv('./2223_player_statistics/Ligue1_player_2223.csv')